# Personalising Spotify through a recommendation engine

## Imports

In [1]:
import pandas as pd
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import DBSCAN
import scipy as sp

pd.options.mode.chained_assignment = None
#gets rid of annoying false-positive warning


In [2]:
%matplotlib inline
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option("display.max_rows", None)

In [3]:
# import sqlite3
# try:
#     conn = sqlite3.connect("million_songs.db")
# except Exception as e:
#     print(e)

# # Now in order to read in pandas dataframe we need to know table name
# cursor = conn.cursor()
# songs_df = pd.read_sql_query('SELECT * FROM FEATURES_2', conn)
# many_genres_df = pd.read_sql_query('SELECT * FROM TRAINING_DB_2', conn)
# conn.close()

# all_songs_df = pd.read_csv('data/songs.csv').head(100) #making it a small number so i can manage lol

In [4]:
def generate_dataframes(library_songs, wasabia_songs):

    library_songs_df = pd.DataFrame(library_songs)
    wasabia_songs_df = pd.DataFrame(wasabia_songs)

    library_songs_df = library_songs_df.drop_duplicates(subset=['id'])
    wasabia_songs_df = wasabia_songs_df.drop_duplicates(subset=['id'])

    library_songs_length = len(library_songs_df.index)
    wasabia_songs_length = len(wasabia_songs_df.index)

    return library_songs_df, wasabia_songs_df, library_songs_length, wasabia_songs_length

In [5]:

with open('personalise_test.json') as json_file:
	test_data_json = json.load(json_file)

library_songs_df, wasabia_songs_df, library_songs_length, wasabia_songs_length = generate_dataframes(test_data_json['wasabia' ], test_data_json['user_playlist'])

display(library_songs_df.head())
display(wasabia_songs_df.head())

,added_at,is_local,track,genres,release_date,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,2022-04-29T15:40:01Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[instrumental math rock, japanese jazz fusion,...",2009-09-02,Waltz For Lilly,0.269,0.662,9,-8.142,1,0.0410,0.000257,0.893,0.0799,0.557,173.257,4rwQ3Rt2Cxm9dlibRhoaTV,265320,3
1,2022-04-29T15:40:01Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[british post-rock, instrumental post-rock, in...",2009-01-06,Set Guitars to Kill,0.160,0.900,0,-5.343,0,0.0608,0.000052,0.844,0.3390,0.476,97.092,7vFft9G8FTZwsBJ98a7Bft,329587,4
2,2022-04-29T15:40:01Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[emo, indie rock, screamo]",2017-08-18,451,0.422,0.802,2,-5.129,0,0.0496,0.000335,0.454,0.1130,0.345,131.900,7tEGdIdX1ocggOYeOYOnIU,292880,4
3,2022-04-29T15:40:01Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[instrumental math rock, instrumental rock, ma...",2007-12-08,"When the Catholic Girls Go Camping, the Nicoti...",0.319,0.750,4,-4.860,0,0.0374,0.000018,0.884,0.0863,0.380,138.112,6svSukdIhpKDFaFJNrblJ4,232987,3
4,2022-04-29T15:40:01Z,False,"{'album': {'images': [{'height': 640, 'url': '...",[instrumental math rock],2021-01-29,The Great Calamity,0.440,0.897,6,-5.322,0,0.0539,0.000121,0.772,0.0608,0.387,137.601,7EPCNK1WiKFbUIhoHuEqii,329000,4


,added_at,is_local,track,genres,release_date,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature
0,2020-07-28T11:36:31Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[aesthetic rap, phonk]",2018-01-08,Evil Fantasy,0.855,0.796,5,-5.976,1,0.3160,0.220,0.000321,0.107,0.655,80.028,3A4TIpLeMng0Nn0N7yre4g,114129,4
1,2020-08-26T12:06:40Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[corrido, corridos tumbados, nueva musica mexi...",2020-07-17,Romance,0.612,0.484,10,-8.161,1,0.0302,0.637,0.000000,0.129,0.524,97.417,6obSwTmKqCqMIFMeqUo6Yv,274215,3
2,2020-07-28T11:36:14Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[lo-fi jazzhop, christian lo-fi]",2019-02-26,no stress,0.615,0.725,0,-6.574,1,0.0362,0.117,0.126000,0.458,0.741,90.001,7ouNolPevRZcmf7wnp0Ypu,229333,4
3,2020-07-28T11:36:59Z,False,"{'album': {'images': [{'height': 640, 'url': '...","[indie pop, vancouver indie]",2007-05-08,Ball Cap,0.470,0.200,7,-10.854,1,0.0362,0.862,0.000015,0.151,0.220,126.927,5tLCG0E1wv5GG3eWSSVU3t,200253,4
4,2020-07-28T11:35:39Z,False,"{'album': {'images': [{'height': 640, 'url': '...",[future funk],2019-02-15,The Day of Night,0.511,0.935,7,-5.417,0,0.5770,0.150,0.730000,0.466,0.438,89.852,1lWoIYZDRuyPEHP3cdo063,74710,4


In [6]:
def generate_all_dataframe(library_songs_df, wasabia_songs_df):
	all_songs_df = pd.concat([library_songs_df, wasabia_songs_df]).reset_index(drop=True)
	all_songs_df['year'] = all_songs_df['release_date'].apply(lambda x: x.split('-')[0])
	return all_songs_df.drop(['release_date', 'is_local', 'track'], axis=1)

In [7]:
all_songs_df = generate_all_dataframe(library_songs_df, wasabia_songs_df)
all_songs_df.tail()

,added_at,genres,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,id,duration_ms,time_signature,year
2668,2022-04-19T14:37:47Z,"[braindance, deep idm, drill and bass, electro...",Brace Yourself Jason,0.546,0.734,7,-8.808,1,0.0343,0.127,0.93200,0.0501,0.447,145.867,4Z8jptUDjJDuxBf0pSOvSi,382373,4,1997
2669,2022-04-19T14:38:05Z,"[folktronica, korean indie rock]",These Chains,0.349,0.774,11,-8.080,1,0.1400,0.583,0.11400,0.4220,0.161,127.900,0KF9n8XeYmrlonaGnL8Krd,307701,4,2019
2670,2022-04-19T14:38:14Z,"[folktronica, korean indie rock]",White Room,0.438,0.631,7,-6.983,0,0.0364,0.390,0.00931,0.1940,0.382,98.810,47kKriL1d9WiVRBy0YxW5V,264067,3,2015
2671,2022-04-19T14:46:33Z,"[permanent wave, pop]",Sparks,0.371,0.268,1,-10.506,1,0.0281,0.748,0.05170,0.1040,0.165,102.617,7D0RhFcb3CrfPuTJ0obrod,227093,3,2000
2672,2022-04-19T14:46:39Z,"[bedroom pop, lo-fi indie]",Feels Like We're Dying,0.579,0.411,6,-13.068,1,0.0329,0.730,0.80000,0.0945,0.125,79.988,0IvaKmBYBLVcdP0Ft8Dwik,185250,1,2017


## Create weight document for genres (one-time run)

In [8]:

def path_to_genre_weights(filepath, min_size):
	with open(filepath) as fp:
		all_genres = fp.read().split('\n')

	genre_range = len(all_genres)
	genre_step = (1-min_size)/genre_range
	return np.arange(min_size, 1, genre_step), all_genres

def only_run_once(genre_scale_min):
	filepath = 'data/genres.txt'
	genre_weights, genre_names = path_to_genre_weights(filepath, min_size=genre_scale_min)
	genre_weights_df = pd.DataFrame(genre_weights, columns=['weight'], index=genre_names)
	genre_weights_df.to_csv('data/genre_weights.csv')

# only_run_once(genre_scale_min=0.5)

In [9]:
def collate_genre_stats():
    genre_weights_df = pd.read_csv('data/genre_weights.csv', index_col=0)
    #grabbin it again bc we need to use it. this code can be re-used lol
    float_cols = all_songs_df.dtypes[all_songs_df.dtypes == 'float64'].index.values
    int_cols = all_songs_df.dtypes[all_songs_df.dtypes == 'int64'].index.values
    return np.concatenate((float_cols, int_cols), axis=None), genre_weights_df

In [10]:
stat_cols, genre_weights_df = collate_genre_stats()

In [11]:
def ohe_prep(df, column): 
    """ 
    Create One Hot Encoded features of a specific column

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        column (str): Column to be processed
        
    Returns: 
        tf_df: One hot encoded features 
    """
    
    tf_df = pd.get_dummies(df[column])
    tf_df.reset_index(drop = True, inplace = True)    
    return tf_df

In [12]:
def create_feature_set(df, stat_cols, genre_weights_df, scale=1):
    """ 
    Process spotify df to create a final set of features that will be used to generate recommendations

    Parameters: 
        df (pandas dataframe): Spotify Dataframe
        stat_cols (list(str)): List of float columns that will be scaled 
        
    Returns: 
        final: final set of features 
    """
    genre_weights_df = genre_weights_df*scale
    song_genre_weights_df = (
        df['genres'].explode()
        .str.get_dummies().sum(level=0)
    )

    for genre in song_genre_weights_df.columns.values:
        song_genre_weights_df[genre] *= genre_weights_df.loc[genre].values[0]

    year_ohe = ohe_prep(df, 'year') * 0.5

    #scale float columns
    floats = df[stat_cols].reset_index(drop = True)
    scaler = MinMaxScaler()
    floats_scaled = pd.DataFrame(scaler.fit_transform(floats), columns = floats.columns) * 0.2

    #concanenate all features

    final = pd.concat([song_genre_weights_df, floats_scaled, year_ohe, df['added_at']], axis = 1).sort_values('added_at',ascending = False)
    # final = pd.concat([song_genre_weights_df, floats_scaled, df['added_at']], axis = 1).sort_values('added_at',ascending = False)
    
    final['id']=df['id'].values
    
    return final.reset_index(drop=True)

In [13]:
feature_set = create_feature_set(all_songs_df, stat_cols, genre_weights_df)
feature_set.head()

C:\Users\ollie\AppData\Local\Temp\ipykernel_27976\318676221.py:14: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  df['genres'].explode()
C:\Users\ollie\AppData\Local\Temp\ipykernel_27976\318676221.py:33: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  final['id']=df['id'].values


,21st century classical,abstract beats,abstract hip hop,acoustic guitar cover,acoustic pop,acoustic punk,acoustic rock,adult standards,aesthetic rap,african-american classical,afrobeat,afrofuturism,afropop,alaska indie,album rock,alt z,alternative dance,alternative emo,alternative hip hop,alternative metal,alternative pop rock,alternative r&b,alternative rock,ambeat,ambient,ambient folk,ambient pop,american 21st century classical,american modern classical,american orchestra,anime,anime lo-fi,anime piano,anime rap,anime rock,anime score,ann arbor indie,anthem emo,anti-folk,antiviral pop,argentine alternative rock,argentine hip hop,argentine indie,argentine indie rock,argentine rock,art pop,art punk,art rock,asian american hip hop,atl hip hop,atl trap,atlanta indie,aussietronica,austindie,australian alternative pop,australian alternative rock,australian dance,australian electropop,australian garage punk,australian indie,australian indie folk,australian pop,australian psych,australian r&b,avant-garde jazz,background jazz,background music,background piano,balalaika,ballet class,baltimore indie,bangalore indie,bangladeshi indie,barbadian pop,barockinterpreten,baroque,baroque ensemble,baroque pop,baroque singing,baroque violin,bases de freestyle,bass trap,bath indie,battle rap,bay area indie,beatlesque,bebop,bedroom pop,bedroom soul,belarusian indie,belgian hip hop,belgian singer-songwriter,bergen indie,big band,big beat,birmingham grime,bitpop,blues rock,bossa nova,bossa nova jazz,boston hip hop,boston indie,boston rock,bow pop,boy band,braindance,brazilian classical,brazilian indie,brazilian jazz,brazilian post-rock,brazilian rock,breakcore,brighton indie,brill building pop,brisbane indie,british alternative rock,british choir,british classical piano,british dance band,british indie rock,british invasion,british jazz,british math rock,british modern classical,british orchestra,british post-rock,british singer-songwriter,british soul,britpop,bronx hip hop,brooklyn drill,brooklyn indie,brostep,bubblegrunge,bubblegum pop,canadian americana,canadian contemporary country,canadian contemporary r&b,canadian electropop,canadian hip hop,canadian indie,canadian metal,canadian modern jazz,canadian pop,canadian psychedelic,canadian rock,canadian singer-songwriter,canadian trap,candy pop,caucasian classical piano,cello,chakra,chamber choir,chamber orchestra,chamber pop,chamber psych,channel islands indie,chanson,charlotte nc indie,chicago indie,chicago punk,chicago rap,chicago soul,chilean indie,chill beats,chill guitar,chill r&b,chillhop,chillsynth,chillwave,chinese hip hop,chinese traditional,chiptune,choral,christian lo-fi,city pop,classic anime,classic canadian rock,classic city pop,classic country pop,classic garage rock,classic girl group,classic j-pop,classic j-rock,classic japanese jazz,classic rock,classic soul,classic soundtrack,classical,classical accordion,classical cello,classical clarinet,classical countertenor,classical era,classical guitar,classical organ,classical performance,classical piano,classify,collage pop,colombian pop,comedy rap,comic,complextro,compositional ambient,connecticut indie,conscious hip hop,contemporary choir,contemporary jazz,contemporary post-bop,contemporary r&b,contemporary vocal jazz,cool jazz,corrido,corridos tumbados,cosmic post-rock,country rock,cowboy western,cyberpunk,czech electronic,dance pop,dance rock,dance-punk,dancehall,dancehall queen,danish pop,dark cabaret,dark pop,dark post-punk,dark synthpop,dark trap,dark wave,darksynth,dc indie,deathcore,deconstructed club,deep adult standards,deep east coast hip hop,deep gothic post-punk,deep idm,denpa-kei,denton tx indie,desert blues,digital hardcore,disco,disco house,disney,dixieland,diy emo,djent,dmv rap,doo-wop,double drumming,doujin,downtempo,downtempo bass,dream pop,dream smp,dreamo,dreampunk,drift,drift phonk,drill,drill and bass,drone,dunedin indie,dutch dnb,dutch pop,early modern classical,early music,early music choir,early music ensemble,e

In [14]:
def summarise_playlist(playlist_df, weight_decay, not_playlist=False):
    """
    Summarize a user's playlist into a single vector

    Parameters: 
        playlist_df (pandas dataframe): playlist dataframe
        weight_factor (float): float value that represents the recency bias. The larger the recency bias, the most priority recent songs get. Value should be close to 1. 
        
    Returns: 
        playlist_feature_set_weighted_final (pandas series): single feature that summarizes the playlist
        complete_feature_set_nonplaylist (pandas dataframe): 
    """
    if not_playlist:
        return playlist_df.sum(axis = 0).drop('id')
    else:
        playlist_feature_set = playlist_df.sort_values('added_at',ascending=False)
        most_recent_date = playlist_feature_set['added_at'][0]
        for ix, row in playlist_feature_set.iterrows():
            playlist_feature_set.loc[ix,'months_from_recent'] = int((pd.to_datetime(most_recent_date) - pd.to_datetime(row['added_at'])).days / 30)
        playlist_feature_set['weight'] = playlist_feature_set['months_from_recent'].apply(lambda x: weight_decay ** (-x))
        playlist_feature_set_weighted = playlist_feature_set.copy()
        playlist_feature_set_weighted.update(playlist_feature_set_weighted.iloc[:,:-4].mul(playlist_feature_set_weighted.weight,0))
        playlist_feature_set_weighted_final = playlist_feature_set_weighted.iloc[:, :-4]
    return playlist_feature_set_weighted_final.mean(axis = 0)

In [15]:
def find_cosine_similarity(playlist_summary, wasabia_features):
    """ 
    Pull songs from a specific playlist.

    Parameters: 
        playlist_summary (pandas series): summarized playlist feature
        wasabia_features (pandas dataframe): feature set of songs that are not in the selected playlist
        
    Returns: 
        non_playlist_df_top_40: Top 40 recommendations for that playlist
    """
    
    wasabia_df = wasabia_features[['id']]
    wasabia_df['sim'] = cosine_similarity(wasabia_features.drop('id', axis = 1).values, playlist_summary.values.reshape(1, -1))[:,0]
    non_playlist_df_top_20 = wasabia_df.sort_values('sim',ascending = False)
    # non_playlist_df_top_40['url'] = non_playlist_df_top_40['id'].apply(lambda x: sp.track(x)['album']['images'][1]['url'])
    
    return non_playlist_df_top_20

In [16]:
def find_similarity(feature_set, library_songs_length, wasabia_songs_length, wasabia_songs_df):
	library_features_df = feature_set.head(library_songs_length)
	wasabia_features_df = feature_set.tail(wasabia_songs_length).drop(['added_at'], axis = 1)

	summarised_library_features = summarise_playlist(library_features_df, 1.09)

	wasabia_cosine_similarity = find_cosine_similarity(summarised_library_features, wasabia_features_df)

	wasabia_songs_df = wasabia_songs_df.set_index('id')

	wasabia_songs_df['sim'] = wasabia_cosine_similarity.set_index('id')['sim']

	return wasabia_songs_df.sort_values('sim',ascending = False)

In [17]:
find_similarity(feature_set, library_songs_length, wasabia_songs_length, wasabia_songs_df).drop(['added_at', 'is_local', 'track', 'release_date'], axis=1)

,genres,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,sim
id,,,,,,,,,,,,,,,,
75ai0ibymmqyCyaxwZuhnu,"[k-pop, k-rap, korean r&b]","I Am You, You Are Me",0.7680,0.61800,9,-6.928,0,0.0529,0.317000,0.000000,0.2990,0.7000,96.975,213252,4,0.648524
4upw9RT8Cs96EQhCuoNryV,"[indie garage rock, indie surf, san diego indi...",You Are Going to Hate This,0.5650,0.76200,9,-7.713,1,0.0309,0.005460,0.169000,0.6230,0.6300,99.932,186559,4,0.642410
3vk4V4vwotEvP16U1cM0HQ,"[bitpop, j-idol, j-pop, j-pop girl group, japa...",エレクトロ・ワールド,0.5130,0.86400,2,-3.364,1,0.0493,0.018700,0.000005,0.1360,0.1150,143.297,261720,4,0.642019
6jhzO1h1C36wr5XztiEN2O,[indie game soundtrack],Sneaky Driver,0.6110,0.66900,6,-6.893,1,0.0297,0.026100,0.706000,0.1120,0.6220,102.951,279853,4,0.631564
6kYCJFQujIt0053wuYnrFe,"[art pop, canadian electropop, dance pop, grav...",Visiting Statue,0.8030,0.63700,10,-11.316,0,0.0447,0.490000,0.876000,0.0959,0.7720,140.016,118853,4,0.626242
1qhprBeThHcoqE8h7OZVez,"[lo-fi beats, lo-fi rap]",starstruck,0.7000,0.08920,1,-14.399,1,0.0405,0.247000,0.724000,0.0615,0.2650,140.032,161176,4,0.625442
5abubE0dYFaEPDeM2sZ4TY,"[dance-punk, industrial metal, modern rock]",What Do They Know?,0.6450,0.79100,7,-5.480,1,0.0927,0.040600,0.000000,0.2930,0.6010,153.952,188573,4,0.623771
02846iBEHz7g1pFLvpRllW,"[j-poprock, japanese alternative rock]",MUKANJYO,0.3160,0.96100,7,-4.082,1,0.1290,0.003640,0.000000,0.2260,0.3040,94.858,192400,4,0.619309
0rCqnLURgOStooTxjORptm,"[lo-fi beats, sad lo-fi]",Imperial,0.3830,0.45600,3,-7.863,1,0.2390,0.236000,0.082000,0.4830,0.4000,102.313,93659,4,0.616856


In [18]:

# library_features_df = feature_set.head(library_songs_length).drop(['added_at'], axis = 1)
# wasabia_features_df = feature_set.tail(wasabia_songs_length).drop(['added_at'], axis = 1)
# db = DBSCAN(eps=0.5, min_samples=5).fit(library_features_df.drop('id', axis = 1).values)
# def dbscan_predict(dbscan_model, X_new, metric=sp.spatial.distance.cosine):
#     # Result is noise by default
#     y_new = np.ones(shape=len(X_new), dtype=int)*-1 
#     # Iterate all input samples for a label
#     for j, x_new in enumerate(X_new):
#         # Find a core sample closer than EPS
#         for i, x_core in enumerate(dbscan_model.components_): 
#             if metric(x_new, x_core) < dbscan_model.eps:
#                 # Assign label of x_core to x_new
#                 y_new[j] = dbscan_model.labels_[dbscan_model.core_sample_indices_[i]]
#                 break
#     return y_new

# wasabia_df = wasabia_features_df[['id']]
# wasabia_cluster_match = dbscan_predict(db, wasabia_features_df.drop('id', axis = 1).values)
# wasabia_songs_df = wasabia_songs_df.set_index('id')
# wasabia_songs_df['cluster'] = wasabia_cluster_match
# wasabia_songs_df.sort_values('cluster',ascending = False).drop(['added_at', 'is_local', 'release_date', 'danceability', 'energy', 'key', 'loudness', 'mode',
#                                        'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature'], axis=1)

Note to self:

Playlist -> Tracks 

-> Get features

-> Get artists -> Genres

Associate features + genres with track!!